In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

print(img_height)
print(img_width)
print(channel)

import random

ind_list = [i for i in range(x_train.shape[0])]

random.seed(1980)
random.shuffle(ind_list)
x_train  = x_train[ind_list, :,:,:]
y_train = y_train[ind_list,]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 39s 0us/step
32
32
3


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
import random

num_filter = 64
dropout_rate = 0.2
l = 10
epochs = 40

random.seed(1980)
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 391s 8ms/step - loss: 1.4025 - acc: 0.4882 - val_loss: 1.9285 - val_acc: 0.4713
Epoch 2/40
19200/50000 [==========>...................] - ETA: 3:33 - loss: 1.0207 - acc: 0.6358

50000/50000 [==============================] - 375s 7ms/step - loss: 0.9512 - acc: 0.6599 - val_loss: 1.1738 - val_acc: 0.6213
Epoch 3/40
45440/50000 [==========================>...] - ETA: 31s - loss: 0.7679 - acc: 0.7277

50000/50000 [==============================] - 375s 7ms/step - loss: 0.7623 - acc: 0.7303 - val_loss: 1.1198 - val_acc: 0.6619
Epoch 4/40
50000/50000 [==============================] - 375s 7ms/step - loss: 0.6475 - acc: 0.7725 - val_loss: 1.8711 - val_acc: 0.5752
Epoch 5/40
 3200/50000 [>.............................] - ETA: 5:26 - loss: 0.6028 - acc: 0.7900

50000/50000 [==============================] - 375s 7ms/step - loss: 0.5685 - acc: 0.8002 - val_loss: 1.3158 - val_acc: 0.6497
Epoch 6/40
39424/50000 [======================>.......] - ETA: 1:13 - loss: 0.5070 - acc: 0.8217

50000/50000 [==============================] - 374s 7ms/step - loss: 0.5071 - acc: 0.8222 - val_loss: 0.9926 - val_acc: 0.7195
Epoch 7/40
50000/50000 [==============================] - 375s 7ms/step - loss: 0.4571 - acc: 0.8407 - val_loss: 0.8727 - val_acc: 0.7555
Epoch 8/40
 1792/50000 [>.............................] - ETA: 5:36 - loss: 0.4268 - acc: 0.8438

...

50000/50000 [==============================] - 375s 7ms/step - loss: 0.0896 - acc: 0.9674 - val_loss: 1.0358 - val_acc: 0.8140
Epoch 34/40
50000/50000 [==============================] - 375s 7ms/step - loss: 0.0853 - acc: 0.9696 - val_loss: 0.9309 - val_acc: 0.8222
Epoch 35/40
 1408/50000 [..............................] - ETA: 5:37 - loss: 0.0703 - acc: 0.9766

50000/50000 [==============================] - 375s 7ms/step - loss: 0.0830 - acc: 0.9697 - val_loss: 0.8898 - val_acc: 0.8303
Epoch 36/40
38528/50000 [======================>.......] - ETA: 1:19 - loss: 0.0832 - acc: 0.9710

50000/50000 [==============================] - 375s 7ms/step - loss: 0.0846 - acc: 0.9704 - val_loss: 0.6134 - val_acc: 0.8697
Epoch 37/40
50000/50000 [==============================] - 375s 7ms/step - loss: 0.0768 - acc: 0.9724 - val_loss: 0.7970 - val_acc: 0.8446
Epoch 38/40
 1408/50000 [..............................] - ETA: 5:38 - loss: 0.0671 - acc: 0.9780

50000/50000 [==============================] - 375s 7ms/step - loss: 0.0759 - acc: 0.9724 - val_loss: 0.6463 - val_acc: 0.8689
Epoch 39/40
38528/50000 [======================>.......] - ETA: 1:19 - loss: 0.0683 - acc: 0.9752

50000/50000 [==============================] - 374s 7ms/step - loss: 0.0723 - acc: 0.9740 - val_loss: 0.8055 - val_acc: 0.8533
Epoch 40/40
50000/50000 [==============================] - 375s 7ms/step - loss: 0.0722 - acc: 0.9740 - val_loss: 0.9855 - val_acc: 0.8366


In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 34s 3ms/step
Test loss: 0.9855307527542114
Test accuracy: 0.8366


In [14]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True)

datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=int(x_train.shape[0]/batch_size),
                    epochs= 3,
                    validation_data=(x_test, y_test))


Epoch 1/3
390/390 [==============================] - 378s 970ms/step - loss: 0.3660 - acc: 0.8796 - val_loss: 0.5366 - val_acc: 0.8605
Epoch 2/3
284/390 [====================>.........] - ETA: 1:35 - loss: 0.2899 - acc: 0.9002

390/390 [==============================] - 378s 969ms/step - loss: 0.2922 - acc: 0.9001 - val_loss: 0.7140 - val_acc: 0.8190
Epoch 3/3
390/390 [==============================] - 379s 971ms/step - loss: 0.2689 - acc: 0.9072 - val_loss: 0.6291 - val_acc: 0.8317


In [15]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 33s 3ms/step
Test loss: 0.6291333000421524
Test accuracy: 0.8317


In [14]:
import os
print( os.getcwd() )
print( os.listdir() )

/content
['datalab', '.keras', '.nv', '.config', '.local', '.ipython', '.forever', '.rnd', '.cache']


In [15]:
# Save the trained weights in to .h5 format
model.save_weights("VK_ASSIGNMENT4B_DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
from google.colab import files

files.download('VK_ASSIGNMENT4B_DNST_model.h5')
print('complete')

complete


In [17]:
# Saving using PyDrive to google drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'VK_ASSIGNMENT4B_DNST_model.h5'})
uploaded.SetContentFile('VK_ASSIGNMENT4B_DNST_model.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
#downloaded = drive.CreateFile({'id': uploaded.get('id')})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1fvCE4F6TmWvzQgzYj60q02Kw061pQzz9
